In [25]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

## Default Values

In [26]:
# Define path to files
path = "C:\\Users\\Asus\\Documents\\Python for Finance\\Python"

# Start date defaults
S_YEAR = 2019
S_MONTH = 1
S_DAY = 1
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2023
E_MONTH = 6
E_DAY = 30
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

## Get Stock File Names in a List

In [27]:
# listdir returns all files in the directory and isfile will return true
# if it is a file and then we store its name in our list named files
files = [x for x in listdir(path) if isfile(join(path, x))]

# Remove extension from file names
# Splitext splits the file name into 2 parts being the name and extension
# We say get all file names and then store just the name in our list named files
tickers = [os.path.splitext(x)[0] for x in files]
tickers


['AAPL',
 'ABNB',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'ALGN',
 'AMAT',
 'AMGN',
 'AMZN',
 'ANSS',
 'ASML',
 'ATVI',
 'AZN',
 'BKNG',
 'BKR',
 'CHTR',
 'CMCSA',
 'COOP',
 'COST',
 'CPRT',
 'CRWD',
 'CSGP',
 'CSX',
 'CTAS',
 'CTSH',
 'DDOG',
 'DLTR',
 'DXCM',
 'EA',
 'EBAY',
 'ENPH',
 'EXC',
 'FANG',
 'FAST',
 'FTNT',
 'GILD',
 'GLDI',
 'GOOG',
 'GOOGL',
 'HON',
 'IDXX',
 'ILMN',
 'INTC',
 'INTU',
 'ISRG',
 'JD',
 'KDP',
 'KHC',
 'KLAC',
 'LCID',
 'LRCX',
 'LULU',
 'MAR',
 'MCHP',
 'MDLZ',
 'MELI',
 'META',
 'MRNA',
 'MRVL',
 'MSFT',
 'MSTR',
 'MU',
 'NFLX',
 'NVDA',
 'NXPI',
 'ODFL',
 'ORLY',
 'PANW',
 'PAYX',
 'PCAR',
 'PDD',
 'PEP',
 'QCOM',
 'REGN',
 'ROST',
 'SBUX',
 'SGEN',
 'SIRI',
 'SNPS',
 'SPLK',
 'TEAM',
 'TMUS',
 'TSLA',
 'TXN',
 'VRSK',
 'VRTX',
 'WBA',
 'WBD',
 'WDAY',
 'WDC',
 'XEL',
 'ZM',
 'ZS']

## Create a Dataframe from our List

In [28]:
stock_df = pd.DataFrame(tickers,columns=['Ticker'])
stock_df

,Ticker
0,AAPL
1,ABNB
2,ADBE
3,ADI
4,ADP
...,...
89,WDAY
90,WDC
91,XEL
92,ZM


## Function that Returns a Dataframe from a CSV

In [29]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv("C:\\Users\\Asus\\Documents\\Python for Finance\\Python\\" + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

## Function that Saves Dataframe to CSV

In [30]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(path + ticker + '.csv')


## Delete Unnamed Columns in CSV Files

In [31]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

## Add Daily Return to Dataframe

In [32]:
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    df.to_csv(path + ticker + '.csv')
    return df


## Returns Return on Investment over Time

In [33]:
# Return on Investment is the return you received from your investment
# This amount does not include your initial investment
# If you invest 100 and have 200 after 5 years
# ROI = End Value (200) - Initial Value (100) / Inital Value = 1
# Your new total is Inital Investment + 1 * Inital Investment = 200

def get_roi_defined_time(df):
    # Set as a datetime
    df['Date'] = pd.to_datetime(df['Date'])
    if 'Date' in df.columns:
        mask = df['Date'] == S_DATE_STR
        if mask.any():
            start_val = df.loc[mask, 'Adj Close'].values[0]
        else:
            # Handle the case when no rows match the date condition
            start_val = None  # or any default value you prefer
    else:
        # Handle the case when 'Date' column does not exist in the DataFrame
        start_val = None  # or any default value you prefer

    print("Initial Price :", start_val)
        
    end_val = df[df['Date'] == E_DATE_STR]['Adj Close']
    print(end_val.item())
    print("Final Price :", end_val.item())
        
    # Calculate return on investment
    roi = (end_val - start_val) / start_val
    
    # Return the total return between 2 dates
    return roi


## Get Coefficient of Variation

In [34]:
# Receives the dataframe with the Adj Close data and returns the coefficient of variation
def get_cov(stock_df):
    mean = stock_df['Adj Close'].mean()
    sd = stock_df['Adj Close'].std()
    cov = sd / mean
    return cov

## Test Functions

In [35]:
# Create a backup for all original stock data

# Get our 1st ticker
tickers[0]

# Get a dataframe for that ticker
stock_a = get_df_from_csv(tickers[0])
stock_a

# Add daily return to this dataframe
add_daily_return_to_df(stock_a, tickers[0])

# Delete unnamed columns in dataframe
stock_a = delete_unnamed_cols(stock_a)
stock_a

# Save cleaned dataframe to csv
save_dataframe_to_csv(stock_a, tickers[0])

## Add Daily Returns & Clean Up All Files

In [36]:
# Create a backup for all original stock data

# Cycle through all tickers
for ticker in tickers:
    print("Working on :", ticker)
    
    # Get a dataframe for that ticker
    stock_df = get_df_from_csv(ticker)
    
    # Add daily return to this dataframe
    add_daily_return_to_df(stock_df, ticker)
    
    # Delete unnamed columns in dataframe
    stock_df = delete_unnamed_cols(stock_df)
    
    # Save cleaned dataframe to csv
    save_dataframe_to_csv(stock_df, ticker)

Working on : AAPL
Working on : ABNB
Working on : ADBE
Working on : ADI
Working on : ADP
Working on : ADSK
Working on : ALGN
Working on : AMAT
Working on : AMGN
Working on : AMZN
Working on : ANSS
Working on : ASML
Working on : ATVI
Working on : AZN
Working on : BKNG
Working on : BKR
Working on : CHTR
Working on : CMCSA
Working on : COOP
Working on : COST
Working on : CPRT
Working on : CRWD
Working on : CSGP
Working on : CSX
Working on : CTAS
Working on : CTSH
Working on : DDOG
Working on : DLTR
Working on : DXCM
Working on : EA
Working on : EBAY
Working on : ENPH
Working on : EXC
Working on : FANG
Working on : FAST
Working on : FTNT
Working on : GILD
Working on : GLDI
Working on : GOOG
Working on : GOOGL
Working on : HON
Working on : IDXX
Working on : ILMN
Working on : INTC
Working on : INTU
Working on : ISRG
Working on : JD
Working on : KDP
Working on : KHC
Working on : KLAC
Working on : LCID
Working on : LRCX
Working on : LULU
Working on : MAR
Working on : MCHP
Working on : MDLZ
Work